## Finding Potentially Useful Tables
First I want to go through the AACT Database Schema table definitions in `aact_tables.xlsx` to identify which tables might contain useful features. After identifying a table, I'll take a look at it's features and select any as potential candidates for features to train the ML models on.

In [2]:
import numpy as np
import pandas as pd
import pathlib

raw_dir = pathlib.Path('../data/raw')

### `calculated_values`
An AACT-provided table that contains info that's been calculated from the information received from ClinicalTrials.gov.  For example, number_of_facilities and actual_duration are provided in this table.

In [3]:
calc_vals = pd.read_csv(raw_dir/'calculated_values.txt', sep='|')

In [4]:
calc_vals.head(10)

,id,nct_id,number_of_facilities,number_of_nsae_subjects,number_of_sae_subjects,registered_in_calendar_year,nlm_download_date,actual_duration,were_results_reported,months_to_report_results,has_us_facility,has_single_facility,minimum_age_num,maximum_age_num,minimum_age_unit,maximum_age_unit,number_of_primary_outcomes_to_measure,number_of_secondary_outcomes_to_measure,number_of_other_outcomes_to_measure
0,170308915,NCT06272461,1,NaN,NaN,2024,NaN,NaN,f,NaN,f,t,18.0,90.0,year,year,1.0,5.0,NaN
1,170308916,NCT02497274,0,NaN,NaN,2015,NaN,37.0,f,NaN,NaN,NaN,18.0,55.0,year,year,1.0,1.0,2.0
2,170308917,NCT05412550,1,NaN,NaN,2022,NaN,NaN,f,NaN,t,t,40.0,89.0,year,year,1.0,7.0,1.0
3,170308918,NCT05292352,1,NaN,NaN,2022,NaN,NaN,f,NaN,t,t,6.0,9.0,year,year,2.0,12.0,NaN
4,170308919,NCT05866458,12,NaN,NaN,2023,NaN,NaN,f,NaN,f,f,50.0,NaN,year,NaN,1.0,4.0,NaN
5,170532907,NCT03632941,1,17.0,4.0,2018,NaN,55.0,t,11.0,t,t,18.0,NaN,year,NaN,1.0,1.0,1.0
6,170532908,NCT04219826,22,NaN,NaN,2020,NaN,38.0,f,NaN,t,f,18.0,85.0,year,year,1.0,7.0,NaN
7,170532909,NCT04527887,1,NaN,NaN,2020,NaN,38.0,f,NaN,t,t,18.0,NaN,year,NaN,1.0,7.0,NaN
8,170532910,NCT06652464,1,NaN,NaN,2024,NaN,3.0,f,NaN,f,t,18.0,80.0,year,year,1.0,1.0,NaN
9,170532911,NCT05578898,1,NaN,NaN,2022,NaN,23.0,f,NaN,t,t,18.0,NaN,year,NaN,5.0,3.0,NaN


In [5]:
# check if only one entry per study
calc_vals.shape[0] == calc_vals['nct_id'].unique().size

True

In [6]:
calc_vals['maximum_age_unit'].value_counts()

maximum_age_unit
year      287149
month       4946
day         2213
week        1793
hour         610
minute       135
Name: count, dtype: int64

In [7]:
calc_vals[calc_vals['maximum_age_unit'] == 'month'].head()

,id,nct_id,number_of_facilities,number_of_nsae_subjects,number_of_sae_subjects,registered_in_calendar_year,nlm_download_date,actual_duration,were_results_reported,months_to_report_results,has_us_facility,has_single_facility,minimum_age_num,maximum_age_num,minimum_age_unit,maximum_age_unit,number_of_primary_outcomes_to_measure,number_of_secondary_outcomes_to_measure,number_of_other_outcomes_to_measure
62,170308944,NCT05994742,5,NaN,NaN,2023,NaN,NaN,f,NaN,f,f,6.0,59.0,month,month,1.0,5.0,8.0
226,170309044,NCT00369759,38,NaN,NaN,2006,NaN,22.0,f,NaN,t,f,1.0,12.0,day,month,1.0,4.0,NaN
370,170309097,NCT02173951,1,NaN,NaN,2014,NaN,NaN,f,NaN,f,t,6.0,36.0,month,month,1.0,1.0,NaN
555,170533243,NCT05973812,3,NaN,NaN,2023,NaN,19.0,f,NaN,f,f,3.0,3.0,month,month,3.0,2.0,NaN
599,170533287,NCT03615495,12,NaN,NaN,2018,NaN,61.0,f,NaN,t,f,NaN,12.0,NaN,month,1.0,NaN,NaN


In [ ]:
# number of studies where min age units doesn't match max age units
calc_vals[
    (calc_vals['maximum_age_unit'] != calc_vals['minimum_age_unit'])
    & ~(calc_vals['maximum_age_unit'].isna() != calc_vals['minimum_age_unit'].isna())
    & ~(calc_vals['maximum_age_unit'].isna() & calc_vals['minimum_age_unit'].isna())
][['minimum_age_unit', 'maximum_age_unit']].shape[0]

7386

In [ ]:
# alot of nans for units so if I want to use, i'll need to decide on how to impute
chosen_units = ['day', 'week', 'month', 'year']
min_age_unit = calc_vals['minimum_age_unit']
max_age_unit = calc_vals['maximum_age_unit']
calc_vals[min_age_unit.isin(chosen_units + [np.nan]) & max_age_unit.isin(chosen_units + [np.nan])]

,id,nct_id,number_of_facilities,number_of_nsae_subjects,number_of_sae_subjects,registered_in_calendar_year,nlm_download_date,actual_duration,were_results_reported,months_to_report_results,has_us_facility,has_single_facility,minimum_age_num,maximum_age_num,minimum_age_unit,maximum_age_unit,number_of_primary_outcomes_to_measure,number_of_secondary_outcomes_to_measure,number_of_other_outcomes_to_measure
0,170308915,NCT06272461,1,NaN,NaN,2024,NaN,NaN,f,NaN,f,t,18.0,90.0,year,year,1.0,5.0,NaN
1,170308916,NCT02497274,0,NaN,NaN,2015,NaN,37.0,f,NaN,NaN,NaN,18.0,55.0,year,year,1.0,1.0,2.0
2,170308917,NCT05412550,1,NaN,NaN,2022,NaN,NaN,f,NaN,t,t,40.0,89.0,year,year,1.0,7.0,1.0
3,170308918,NCT05292352,1,NaN,NaN,2022,NaN,NaN,f,NaN,t,t,6.0,9.0,year,year,2.0,12.0,NaN
4,170308919,NCT05866458,12,NaN,NaN,2023,NaN,NaN,f,NaN,f,f,50.0,NaN,year,NaN,1.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559366,170308911,NCT03327467,2,NaN,NaN,2017,NaN,NaN,f,NaN,t,f,NaN,26.0,NaN,year,NaN,NaN,NaN
559367,170308912,NCT06950593,1,NaN,NaN,2025,NaN,NaN,f,NaN,t,t,18.0,NaN,year,NaN,5.0,3.0,NaN
559368,170308913,NCT06544694,4,NaN,NaN,2024,NaN,5.0,f,NaN,t,f,18.0,NaN,year,NaN,1.0,NaN,NaN
559369,170308914,NCT07120724,1,NaN,NaN,2025,NaN,NaN,f,NaN,f,t,NaN,NaN,NaN,NaN,3.0,7.0,NaN


In [255]:
# luckily no na units with number
[
    calc_vals[calc_vals['maximum_age_unit'].isna() & ~calc_vals['maximum_age_num'].isna()].shape[0],
    calc_vals[calc_vals['minimum_age_unit'].isna() & ~calc_vals['minimum_age_num'].isna()].shape[0]
]

[0, 0]

- **nsae/sae:** (non) serious adverse event
- Working with age units of minutes and hours seems messy. So I might just include year and month. I might have to calculate age*12 if the age unit is year so that an age_unit column isn't needed.
- Also will need to decide what to do when minimum_age_num is na but maximum_age_num isn't. Might make more sense to impute with 0 instead of the mean.

In [ ]:
# might remove nsae and sae stuff since that wouldn't be known at the beginning of a trial
calc_vals_cols = [
    'nct_id', 'number_of_facilities', 'has_us_facility', 'number_of_nsae_subjects', 'number_of_sae_subjects',
    'minimum_age_num', 'maximum_age_num', 'minimum_age_unit', 'maximum_age_unit',
    'number_of_primary_outcomes_to_measure', 'number_of_secondary_outcomes_to_measure'
    ]

## `central_contacts`
Contact info for people (primary & backup) who can answer questions concerning enrollment at any location of the study.

In [9]:
contacts = pd.read_csv(raw_dir/'central_contacts.txt', sep='|')

In [10]:
contacts.head(10)

,id,nct_id,contact_type,name,phone,email,phone_extension,role
0,56422417,NCT05460416,primary,Julie Collée,+32498973386,julie.collee@uliege.be,NaN,CONTACT
1,56422418,NCT05460416,backup,Marie Timmermans,NaN,marie.timmermans@chuliege.be;,NaN,CONTACT
2,56422419,NCT06791369,primary,"Heinz Jungbluth, MD PhD MRCP MRCPCH",+44 20 71883998,heinz.jungbluth@gstt.nhs.uk,NaN,CONTACT
3,56422420,NCT06791369,backup,"Arti M Mistry, PhD MSci",NaN,arti.mistry@gstt.nhs.uk,NaN,CONTACT
4,56422421,NCT05642156,primary,Alexander H Kirsch,+43316385,alexander.kirsch@medunigraz.at,16023,CONTACT
5,56422422,NCT06793631,primary,"Nicola White, PhD",+44 (0) 2076799057,n.g.white@ucl.ac.uk,NaN,CONTACT
6,56422423,NCT06793631,backup,"Alessandro Bosco, PhD",NaN,alessandro.bosco@ucl.ac.uk,NaN,CONTACT
7,56422424,NCT06796816,primary,"Cristian Rapicetta, MD",0522296858,Cristian.rapicetta@ausl.re.it,NaN,CONTACT
8,56422425,NCT06489301,primary,"Manager, Clinical Research Operations",937-245-7500,pturesearch@wrightstatephysicians.org,NaN,CONTACT
9,56422426,NCT06489301,backup,Regulatory Specialist,937-245-7500,pturesearch@wrightstatephysicians.org,NaN,CONTACT


In [11]:
# more than one entry for each study
contacts.shape[0] == contacts['nct_id'].unique().size

False

In [12]:
contacts['role'].value_counts()
contacts['contact_type'].value_counts()

contact_type
primary    143948
backup      67393
Name: count, dtype: int64

In [ ]:
contacts['nct_id'].nunique() # might significantly lower the number of usable rows

143948

In [13]:
# wondering if not having up to date contact information can effect termination risk
print('contact info | num missing')
for col in contacts.columns:
    print(f"{col}: {contacts[col].isna().sum()}")

contact info | num missing
id: 0
nct_id: 0
contact_type: 0
name: 1
phone: 12965
email: 4816
phone_extension: 185659
role: 0


- might be able to use 4 features for each nct_id: primary_email_missing, backup_email_missing, primary_phone_missing, backup_phone_missing
- will drop the study with the missing name

In [14]:
contacts_cols = [
    'id', 'nct_id', 'contact_type', 'phone', 'email'
]

actually decided not to use contacts for now

## `conditions` and `browse_conditions`
- conditions: Name(s) of the disease(s) or condition(s) studied in the clinical study, or the focus of the clinical study. Can include NLM's Medical Subject Heading (MeSH)-controlled vocabulary terms.
- browse_conditions: NLM uses an internal algorithm to assess the data entered for a study and creates a list of standard MeSH terms that describe the condition(s) being addressed by the clinical trial.  This table provides the results of NLM's assessment

In [15]:
conditions = pd.read_csv(raw_dir/'conditions.txt', sep='|')
browse_conditions = pd.read_csv(raw_dir/'browse_conditions.txt', sep='|')

In [16]:
conditions.head(10)

,id,nct_id,name,downcase_name
0,258969120,NCT02413840,COPD,copd
1,258969121,NCT02413840,Anxiety,anxiety
2,258969122,NCT02413840,Depression,depression
3,258969123,NCT04661215,Gastroparesis,gastroparesis
4,258969124,NCT04661215,Idiopathic Gastric Motility Disorder,idiopathic gastric motility disorder
5,258969125,NCT04661215,Diabetic Gastroparesis,diabetic gastroparesis
6,258969126,NCT06181136,Mucopolysaccharidosis Type IIIA,mucopolysaccharidosis type iiia
7,258969127,NCT04585750,Advanced Solid Tumor,advanced solid tumor
8,258969128,NCT04585750,Advanced Malignant Neoplasm,advanced malignant neoplasm
9,258969129,NCT04585750,Metastatic Cancer,metastatic cancer


In [17]:
browse_conditions.head(10)

,id,nct_id,mesh_term,downcase_mesh_term,mesh_type
0,1014094542,NCT02521727,Intestinal Neoplasms,intestinal neoplasms,mesh-ancestor
1,1014094543,NCT02521727,Gastrointestinal Neoplasms,gastrointestinal neoplasms,mesh-ancestor
2,1014094544,NCT02521727,Digestive System Neoplasms,digestive system neoplasms,mesh-ancestor
3,1014094545,NCT02521727,Neoplasms by Site,neoplasms by site,mesh-ancestor
4,1014094546,NCT02521727,Neoplasms,neoplasms,mesh-ancestor
5,1014094547,NCT02521727,Digestive System Diseases,digestive system diseases,mesh-ancestor
6,1014094548,NCT02521727,Gastrointestinal Diseases,gastrointestinal diseases,mesh-ancestor
7,1014094549,NCT02521727,Colonic Diseases,colonic diseases,mesh-ancestor
8,1014094550,NCT02521727,Intestinal Diseases,intestinal diseases,mesh-ancestor
9,1014094551,NCT02521727,Rectal Diseases,rectal diseases,mesh-ancestor


In [19]:
# I think way too many categories, will need to find a way to group them or something
conditions['downcase_name'].unique().size, conditions.shape[0]

(122341, 990427)

In [ ]:
# mesh terms gives way fewer unique conditions even with more rows
browse_conditions['downcase_mesh_term'].unique().size, browse_conditions.shape[0]

(5996, 4086571)

In [174]:
[cond for cond in conditions['downcase_name'].unique() if 'anxiety' in cond][:10]

['anxiety',
 'anxiety disorders',
 'fear anxiety',
 'anxiety disorder',
 'anxiety in those patients with anorexia nervosa',
 'generalized anxiety disorder',
 'anxiety, preoperative',
 'anxiety postoperative',
 'anxiety depression',
 'pediatric pain and anxiety']

In [110]:
[term for term in browse_conditions['downcase_mesh_term'].unique() if 'anxiety' in term]

['anxiety disorders', 'generalized anxiety disorder', 'anxiety, separation']

In [ ]:
# find which table contains more studies
num_studies_browse = browse_conditions['nct_id'].nunique()
num_studies_conds = conditions['nct_id'].nunique()
print(f"browse_conditions has {num_studies_browse} studies\nconditions has {num_studies_conds} studies")

browse_conditions has 443715 studies
conditions has 558394 studies


In [57]:
# browse_condtions table has fewer studies than conditions table
# still might be worth using if it makes grouping the conditions easier

# find average number of mesh terms each study is associated with
browse_conditions.groupby(['nct_id'])['mesh_term'].nunique().describe()

count    443715.000000
mean          9.209900
std           6.405793
min           1.000000
25%           5.000000
50%           8.000000
75%          12.000000
max         403.000000
Name: mesh_term, dtype: float64

In [33]:
print(len([cond for cond in browse_conditions['downcase_mesh_term'].unique() if 'anxiety' in cond]))
print([cond for cond in browse_conditions['downcase_mesh_term'].unique() if 'anxiety' in cond])

3
['anxiety disorders', 'generalized anxiety disorder', 'anxiety, separation']


Still not sure if I'll try to do some clustering to get a manageable number of groups of mesh terms or If I can just use an LLM to help make a dictionary of groups. Will return to this later.

## `interventions` and `browse_interventions`
- interventions: The interventions or exposures (including drugs, medical devices, procedures, vaccines, and other products) of interest to the study, or associated with study arms/groups.
- browse_intervention: NLM uses an internal algorithm to assess the data entered for a study and creates a list of standard MeSH terms that describe the intervention(s) being addressed by the clinical trial.  This table provides the results of NLM's assessment

Probably just do a similar analysis to conditions and browse_conditions then come back later

In [34]:
interventions = pd.read_csv(raw_dir/'interventions.txt', sep='|')
browse_interventions = pd.read_csv(raw_dir/'browse_interventions.txt', sep='|')

In [35]:
interventions.head()

,id,nct_id,intervention_type,name,description
0,249020644,NCT06432478,DEVICE,3D-printed custom applicator,3D-printed custom applicator
1,249589199,NCT06305156,DIAGNOSTIC_TEST,Bone Mineral Density quantification,"In the first visit, participants receive clini..."
2,249020645,NCT04776928,BEHAVIORAL,Push reports,Surgeons in the experimental arm will receive ...
3,249589200,NCT06023407,OTHER,Water Immersion Heat Therapy,10-weeks of 3 days per week in 1 of the 2 arms
4,249589201,NCT05950022,PROCEDURE,Endoscopic tenotomy surgery of the iliopsoas t...,The surgery will be performed under general an...


In [36]:
browse_interventions.head()

,id,nct_id,mesh_term,downcase_mesh_term,mesh_type
0,491888753,NCT00608335,Lipopeptides,lipopeptides,mesh-ancestor
1,491888754,NCT00608335,Lipids,lipids,mesh-ancestor
2,491888755,NCT00608335,Peptides,peptides,mesh-ancestor
3,491888756,NCT00608335,"Amino Acids, Peptides, and Proteins","amino acids, peptides, and proteins",mesh-ancestor
4,491888757,NCT00608335,Echinocandins,echinocandins,mesh-ancestor


Acutally... interventions already has an intervention_type column which could already be useful...

In [ ]:
# only 11 !
interventions['intervention_type'].unique()

array(['DEVICE', 'DIAGNOSTIC_TEST', 'BEHAVIORAL', 'OTHER', 'PROCEDURE',
       'COMBINATION_PRODUCT', 'DIETARY_SUPPLEMENT', 'BIOLOGICAL', 'DRUG',
       'GENETIC', 'RADIATION'], dtype=object)

In [ ]:
interventions['nct_id'].nunique(), interventions.shape[0] # multiple entries for each study

(503387, 946504)

In [55]:
# how many studies contained in this table?
print("number of studies in interventions table:", interventions['nct_id'].nunique())

# can a study have multiple intervention types? -- yes
interventions.groupby(['nct_id'])['intervention_type'].nunique()

number of studies in interventions table: 503387


nct_id
NCT00000102    1
NCT00000104    1
NCT00000105    2
NCT00000106    1
NCT00000108    1
              ..
NCT07255183    2
NCT07255196    1
NCT07255209    1
NCT07255222    1
NCT07255248    1
Name: intervention_type, Length: 503387, dtype: int64

Will probable just have table with nct_id and 11 columns corresponding to intervention type. all boolean to see if study is associated with intervention type or not

In [ ]:
interventions_cols = ['intervention_type']

## `countries`
Countries in which the study has facilities/sites.

In [62]:
countries = pd.read_csv(raw_dir/"countries.txt", sep="|")
countries.head(10)

,id,nct_id,name,removed
0,202780922,NCT00349622,Puerto Rico,t
1,202780923,NCT02999932,Austria,t
2,202780924,NCT04655183,Canada,t
3,202780925,NCT04655183,United States,t
4,202780926,NCT04105062,United States,t
5,202780927,NCT00825643,United Kingdom,t
6,202780928,NCT01639014,Netherlands,t
7,202780929,NCT02581189,Canada,t
8,202780930,NCT00425906,United States,t
9,202780931,NCT00043407,United States,t


In [64]:
countries['name'].nunique()

228

In [130]:
# maybe could do a num_countries feature to see 
# if having facilities in many countries is correlated with termination risk

print("number of studies countries table has:", countries['nct_id'].nunique())

# not exactly sure what 'removed' means
# i'm assuming it means the facility was removed
# in that case I should only count countries where removed == 'f'
print("\n",countries['removed'].value_counts())
countries[countries['removed'] == 'f'].groupby(['nct_id'])['name'].nunique().describe()

number of studies countries table has: 508511

 removed
f    728940
t     35099
Name: count, dtype: int64


count    502396.000000
mean          1.450756
std           2.412375
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          59.000000
Name: name, dtype: float64

In [100]:
# or not cause this is about risk before a trial starts?
study_country_count = countries.groupby(['nct_id'])['name'].nunique()
study_country_count.describe() # why is min 0?

count    508511.000000
mean          1.502333
std           2.608692
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          61.000000
Name: name, dtype: float64

In [ ]:
# min is 0 because name is NaN
studies_with_0_countries = study_country_count[study_country_count == 0].index.tolist()
countries[[(nct in studies_with_0_countries) for nct in countries['nct_id']]]

,id,nct_id,name,removed
38930,202334131,NCT06640985,NaN,f
376736,202288194,NCT06583473,NaN,f
733270,202166718,NCT01001572,NaN,f


In [ ]:
# could've known that by looking at this
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764039 entries, 0 to 764038
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       764039 non-null  int64 
 1   nct_id   764039 non-null  object
 2   name     763953 non-null  object
 3   removed  764039 non-null  object
dtypes: int64(1), object(3)
memory usage: 23.3+ MB


might remove studies with no value for a country name and just have a column with num_countries

In [238]:
# will include study_country_count as a feature
pd.DataFrame({'nct_id':study_country_count.index.to_numpy(), 'num_countries':study_country_count.values}).tail()

,nct_id,num_countries
508506,NCT07255183,1
508507,NCT07255196,1
508508,NCT07255209,1
508509,NCT07255222,1
508510,NCT07255235,1


## `design_groups`
Defines the protocol-specified group, subgroup, or cohort of participants in a clinical trial assigned to receive specific intervention(s) or observations according to a protocol.

In [134]:
design_groups = pd.read_csv(raw_dir/"design_groups.txt", sep='|')
design_groups.head(10)

,id,nct_id,group_type,title,description
0,277076255,NCT04797429,EXPERIMENTAL,intervention group,The intervention group will receive antidiabet...
1,277076256,NCT04797429,ACTIVE_COMPARATOR,control group,The control group receives the antidiabetic th...
2,277076257,NCT06471595,EXPERIMENTAL,Intervention Group - PLP + SDoH,Participants randomized to the Intervention Gr...
3,277076258,NCT06471595,NO_INTERVENTION,Waitlist Control Group,Participants randomized to the Waitlist Contro...
4,277076259,NCT06112431,EXPERIMENTAL,Intervention Arm,Participants randomized to the Intervention Ar...
5,277076260,NCT06112431,PLACEBO_COMPARATOR,Usual Care,Consented participants randomized to the Usual...
6,277076261,NCT04839081,NaN,RA patients,RA patients diagnosed according to the 1987 AC...
7,277076262,NCT04839081,NaN,Control,Age- and sex-matched control subjects with noc...
8,277076263,NCT06272747,EXPERIMENTAL,Experimental: A (XH-S003),Participants will receive XH-S003 once or twic...
9,277076264,NCT06272747,PLACEBO_COMPARATOR,Placebo Comparator: B (Placebo),Participants will receive matching placebo onc...


In [144]:
# maybe having a lot of groups to handle can effect termination risk?
num_groups_by_study = design_groups.groupby('nct_id')['id'].count()

In [165]:
# just to get an idea of why a study would  have so many groups
ten_groups = num_groups_by_study[num_groups_by_study == 10].index.tolist()
design_groups[[nct in ten_groups for nct in design_groups['nct_id']]].head(20)['title']
# -> different sequences or doses are examples

1189                         Group 1: Sequence 1 (ABC)
1190                         Group 1: Sequence 2 (BCA)
1191                         Group 1: Sequence 3 (CAB)
1192                         Group 1: Sequence 4 (ACB)
1193                         Group 1: Sequence 5 (BAC)
1194                         Group 1: Sequence 6 (CBA)
1205                        Group 2: Sequence 1 (AFDE)
1206                        Group 2: Sequence 2 (DAEF)
1209                        Group 2: Sequence 3 (EDFA)
1210                        Group 2: Sequence 4 (FEAD)
2575               Dose 1 -Single Ascending Dose (SAD)
2576               Dose 2 -Single Ascending Dose (SAD)
2577                Dose 3 -Single Ascending Dose(SAD)
2578                Dose 4 -Single Ascending Dose(SAD)
2579                Dose 5 -Single Ascending Dose(SAD)
2584             Dose 1 - Multiple Ascending Dose(MAD)
2585              Dose 2 -Multiple Ascending Dose(MAD)
2586              Dose 3 -Multiple Ascending Dose(MAD)
2587    Mu

Could have a column for number of groups

Actually realized that number of groups will be from `studies` table

## `designs`
Description of how the study will be conducted, including comparison group design and strategies for masking and allocating participants.

In [168]:
designs = pd.read_csv(raw_dir/"designs.txt", sep='|')
designs.head(10)

,id,nct_id,allocation,intervention_model,observational_model,primary_purpose,time_perspective,masking,masking_description,intervention_model_description,subject_masked,caregiver_masked,investigator_masked,outcomes_assessor_masked
0,145637912,NCT01442324,NaN,SINGLE_GROUP,NaN,TREATMENT,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN
1,145637913,NCT06325397,NaN,NaN,COHORT,NaN,PROSPECTIVE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,145971785,NCT05199571,NaN,SINGLE_GROUP,NaN,TREATMENT,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN
3,145860057,NCT06110676,RANDOMIZED,PARALLEL,NaN,TREATMENT,NaN,QUADRUPLE,NaN,NaN,t,t,t,t
4,145971786,NCT02425345,RANDOMIZED,PARALLEL,NaN,PREVENTION,NaN,SINGLE,NaN,NaN,f,f,f,t
5,145971787,NCT04867837,RANDOMIZED,PARALLEL,NaN,TREATMENT,NaN,DOUBLE,Blinded,NaN,t,f,t,f
6,145971788,NCT05170412,NaN,NaN,COHORT,NaN,CROSS_SECTIONAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,145971789,NCT05379985,NaN,SINGLE_GROUP,NaN,TREATMENT,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN
8,145971790,NCT03367156,RANDOMIZED,PARALLEL,NaN,SUPPORTIVE_CARE,NaN,DOUBLE,NaN,NaN,t,f,t,f
9,145971791,NCT05916313,NaN,SINGLE_GROUP,NaN,TREATMENT,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
print(designs['primary_purpose'].unique())
designs['primary_purpose'].isna().sum(), (~designs['primary_purpose'].isna()).sum()

['TREATMENT' nan 'PREVENTION' 'SUPPORTIVE_CARE' 'OTHER'
 'HEALTH_SERVICES_RESEARCH' 'DIAGNOSTIC' 'BASIC_SCIENCE' 'SCREENING'
 'DEVICE_FEASIBILITY' 'ECT']


(np.int64(134351), np.int64(420311))

In [170]:
designs['intervention_model'].unique()

array(['SINGLE_GROUP', nan, 'PARALLEL', 'SEQUENTIAL', 'CROSSOVER',
       'FACTORIAL'], dtype=object)

In [171]:
designs['observational_model'].unique()

array([nan, 'COHORT', 'CASE_ONLY', 'OTHER', 'CASE_CONTROL',
       'FAMILY_BASED', 'CASE_CROSSOVER', 'ECOLOGIC_OR_COMMUNITY',
       'DEFINED_POPULATION', 'NATURAL_HISTORY'], dtype=object)

In [223]:
designs.shape[0] == designs['nct_id'].nunique()

True

In [224]:
# are there any where both intervention_model and observational_model are NaN?
nan_model = designs[designs['intervention_model'].isna() & designs['observational_model'].isna()]
print("studies that would be dropped:", nan_model.shape[0])
print("remaining studies:", designs[~(designs['intervention_model'].isna() & designs['observational_model'].isna())].shape[0])
# there are, so I might remove those studies.
# for the ml algorithm, I might just have one categorical feature named 'model' 
# which includes intervention and observational model


studies that would be dropped: 9650
remaining studies: 545012


In [187]:
# any studies with both interventional_model and observational_model?
any(~designs['intervention_model'].isna() & ~designs['observational_model'].isna())
# nope, good to go there

False

In [ ]:
# allocation seems less useful
print(designs['allocation'].value_counts())
designs['allocation'].isna().sum()

allocation
RANDOMIZED        280265
NON_RANDOMIZED     45099
Name: count, dtype: int64


np.int64(229298)

In [ ]:
# drop rows where both intervention and observation are nan
# and where primary_purpose is nan
designs_cols = [
    'intervention_model', 'observational_model', 'primary_purpose'
]

## `drop_withdrawals`
Summarized information about how many participants withdrew from the study, when and why. This information explains disposition of participants relative to the numbers starting and completing the study (enumerated in the Milestones table).

In [191]:
pd.read_csv(raw_dir/'drop_withdrawals.txt', sep='|').head()
# includes withdrawals after study started
# so maybe not what I'm looking for in terms of termination risk
# I was kind of more looking towards termination risk when a trial is proposed

,id,nct_id,result_group_id,ctgov_group_code,period,reason,count,drop_withdraw_comment,reason_comment,count_units
0,145717901,NCT00603304,525483064,FG000,Overall Study,Lost to Follow-up,5,NaN,NaN,NaN
1,145717902,NCT00603304,525483065,FG001,Overall Study,Lost to Follow-up,7,NaN,NaN,NaN
2,145717903,NCT00603304,525483064,FG000,Overall Study,Withdrawal by Subject,8,NaN,NaN,NaN
3,145717904,NCT00603304,525483065,FG001,Overall Study,Withdrawal by Subject,5,NaN,NaN,NaN
4,145717905,NCT00603304,525483064,FG000,Overall Study,Protocol Violation,1,NaN,NaN,NaN


## `eligibilities`
Information about the criteria used to select participants; includes inclusion and exclusion criteria

In [193]:
elig = pd.read_csv(raw_dir/"eligibilities.txt", sep='|')
elig.head()

,id,nct_id,sampling_method,gender,minimum_age,maximum_age,healthy_volunteers,population,criteria,gender_description,gender_based,adult,child,older_adult
0,146731333,NCT04817410,NaN,ALL,18 Years,NaN,f,NaN,Inclusion Criteria:~* Emergency Department pat...,NaN,NaN,t,f,t
1,146731334,NCT02002663,NaN,ALL,18 Years,85 Years,f,NaN,Inclusion Criteria:~* Male and females 18-85 y...,NaN,NaN,t,f,t
2,146731335,NCT02818257,NaN,ALL,18 Years,NaN,t,NaN,Inclusion Criteria:~* Have given written infor...,NaN,NaN,t,f,t
3,146731336,NCT05824585,NaN,ALL,18 Years,NaN,f,NaN,Inclusion Criteria:~1. Male or female particip...,NaN,NaN,t,f,t
4,146731337,NCT06775782,NaN,ALL,18 Years,85 Years,f,NaN,Inclusion Criteria~1. Male or female with age ...,NaN,NaN,t,f,t


In [194]:
# has some of same information from calculated_values
# need to consider if it'd be better to have adult/child/older_adult
# instead of actual min age and max age

# actual criteria seems too messy to sift through
# this table might just complicate the model
# only consideration is to use adult/child/older_adult from here
# instead of max and min age from calculated_values

## `studies`
Basic info about study, including study title, date study registered with ClinicalTrials.gov, date results first posted to ClinicalTrials.gov, dates for study start and completion, phase of study, enrollment status, planned or actual enrollment, number of study arms/groups, etc.

In [196]:
studies = pd.read_csv(raw_dir/"studies.txt", sep='|')
studies.head()

/var/folders/4k/08twf02j2fbbssp3kfwnqxv00000gn/T/ipykernel_47206/3429535017.py:1: DtypeWarning: Columns (47,53,68) have mixed types. Specify dtype option on import or set low_memory=False.
  studies = pd.read_csv(raw_dir/"studies.txt", sep='|')


,nct_id,nlm_download_date_description,study_first_submitted_date,results_first_submitted_date,disposition_first_submitted_date,last_update_submitted_date,study_first_submitted_qc_date,study_first_posted_date,study_first_posted_date_type,results_first_submitted_qc_date,...,plan_to_share_ipd_description,created_at,updated_at,source_class,delayed_posting,expanded_access_nctid,expanded_access_status_for_nctid,fdaaa801_violation,baseline_type_units_analyzed,patient_registry
0,NCT06537258,NaN,2024-08-01,NaN,NaN,2025-03-13,2024-08-01,2024-08-05,ACTUAL,NaN,...,Data will be shared with Researchers who provi...,2025-11-30 19:44:24.75252,2025-11-30 19:44:24.75252,OTHER,NaN,NaN,NaN,NaN,NaN,NaN
1,NCT06833801,NaN,2025-02-12,NaN,NaN,2025-02-18,2025-02-13,2025-02-19,ACTUAL,NaN,...,NaN,2025-11-30 19:44:24.75252,2025-11-30 19:44:24.75252,OTHER,NaN,NaN,NaN,NaN,NaN,NaN
2,NCT06790056,NaN,2024-01-11,NaN,NaN,2025-01-22,2025-01-22,2025-01-23,ACTUAL,NaN,...,NaN,2025-11-30 19:44:24.75252,2025-11-30 19:44:24.75252,OTHER,NaN,NaN,NaN,NaN,NaN,f
3,NCT02520063,NaN,2015-07-30,2023-02-07,NaN,2024-12-20,2015-08-06,2015-08-11,ESTIMATED,2023-08-15,...,NaN,2025-11-30 19:44:24.75252,2025-11-30 19:44:24.75252,OTHER,NaN,NaN,NaN,NaN,NaN,NaN
4,NCT01256333,NaN,2010-12-06,NaN,NaN,2013-07-02,2010-12-07,2010-12-08,ESTIMATED,NaN,...,NaN,2025-11-29 20:31:15.134739,2025-11-29 20:31:15.134739,OTHER,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# here's the target
print(studies['nct_id'].nunique() == studies.shape[0])
print("number of potential training & test studies:", studies['overall_status'].isin(['COMPLETED', 'TERMINATED']).sum())

True
number of potential training & test studies: 338720


In [ ]:
# how many studies have phase info out of those either completed or terminated?
print(studies['phase'].unique())
term_or_comp = studies[studies['overall_status'].isin(['COMPLETED', 'TERMINATED'])]
print("sutdies with phase:", (~term_or_comp['phase'].isna()).sum())

[nan 'PHASE1/PHASE2' 'PHASE4' 'PHASE2' 'PHASE2/PHASE3' 'PHASE3' 'PHASE1'
 'EARLY_PHASE1']
sutdies with phase:


np.int64(142408)

In [ ]:
studies_cols = [
    'biospec_description', 'biospec_retention', 'is_ppd', 'phase',
    'is_unapproved', 'is_fda_regulated_device', 'is_fda_regulated_drug',
    'number_of_groups', 'number_of_arms', 'overall_status'
]

True

## `sponsors`
Name of study sponsors and collaborators. The sponsor is the entity or individual initiating the study. Collaborators are other organizations providing support, including funding, design, implementation, data analysis, and reporting.

In [256]:
sponsors = pd.read_csv(raw_dir/"sponsors.txt", sep='|')
sponsors

,id,nct_id,agency_class,lead_or_collaborator,name
0,242825344,NCT06601699,OTHER,collaborator,Tiny Blue Dot Foundation
1,242825345,NCT05350371,OTHER,collaborator,Mayo Clinic
2,242825346,NCT05641987,FED,collaborator,National Institute for Occupational Safety and...
3,242825347,NCT01905566,INDUSTRY,collaborator,AstraZeneca
4,242825348,NCT01905566,OTHER,collaborator,"CardioVascular Research Foundation, Korea"
...,...,...,...,...,...
894972,242466928,NCT02474576,OTHER,lead,Groupe Hospitalier Diaconesses Croix Saint-Simon
894973,242466929,NCT06100146,OTHER,lead,University Ghent
894974,242466930,NCT03497910,NaN,lead,[Redacted]
894975,242466931,NCT00505115,OTHER,lead,"Instituto Valenciano de Infertilidad, IVI VALE..."


In [260]:
print("potentially more than one sponsor per study:", sponsors['nct_id'].nunique() < sponsors.shape[0])
print("number of different sponsors:", sponsors['name'].nunique())
print("number of different agency classes:", sponsors['agency_class'].nunique())

potentially more than one sponsor per study: True
number of different sponsors: 98878
number of different agency classes: 9


In [262]:
sponsors['agency_class'].value_counts()

agency_class
OTHER        572288
INDUSTRY     177808
UNKNOWN       48070
NIH           46161
OTHER_GOV     30924
FED           10634
NETWORK        7299
INDIV           759
AMBIG            90
Name: count, dtype: int64

In [273]:
# each study only has one lead sponsor
print(sponsors['lead_or_collaborator'].unique())
sponsors[sponsors['lead_or_collaborator'] == 'lead'].groupby('nct_id')['id'].count().value_counts()

['collaborator' 'lead']


id
1    559371
Name: count, dtype: int64

In [ ]:
# will filter to only include lead sponsor
sponsor_cols = ['nct_id', 'lead_or_collaborator']